In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
import bs4 as bs
import pickle
import requests
import mpld3
mpld3.enable_notebook()
%matplotlib inline

/home/harsh/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
df_JPM=pd.read_csv('JPM1.csv')

In [3]:
df_JPM.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,SMA,SMA_100,EMA,Gain,Loss,Average_Gain,Average_Loss,Relative_Strength,RSI,CMF
2009,2017-12-22,108.199997,108.459999,106.809998,107.449997,106.893158,14573400,104.662281,96.556956,104.457278,0.000000,0.430000,0.466428,0.373571,1.248565,55.527201,-0.541789
2010,2017-12-26,107.199997,107.989998,106.400002,107.019997,106.465385,7403700,105.114426,96.706751,104.648526,0.200004,0.000000,0.480714,0.317142,1.515767,60.250688,-0.524666
2011,2017-12-27,106.669998,107.290001,106.669998,107.220001,106.664352,9496500,105.405908,96.847064,104.840509,0.570000,0.000000,0.521428,0.295000,1.767555,63.867020,-0.535400
2012,2017-12-28,107.400002,107.839996,107.160004,107.790001,107.231400,7440600,105.607856,96.989486,105.068213,0.000000,0.849999,0.427857,0.355714,1.202812,54.603476,-0.546378
2013,2017-12-29,108.089996,108.279999,106.940002,106.940002,106.385803,8925700,105.728229,97.126815,105.193698,0.000000,0.000000,0.427857,0.333571,1.282655,56.191368,-0.537847


In [8]:
#we will create a temporary dataframe df
df=pd.DataFrame(data=(df_JPM['High']+df_JPM['Low']+df_JPM['Close']/3),columns=['Typical Price'])
df['Raw_Money_Flow']=df['Typical Price']*df_JPM['Volume']
up=[]
down=[]
for index,row in df[1:].iterrows():
    if df.iloc[index]['Typical Price'] >= df.iloc[index-1]['Typical Price']:
        up.append(1)
        down.append(0)
      
    else:
        down.append(1)
        up.append(0)
df_up_down=pd.DataFrame(list(zip(up,down)),columns=['up','down'])
df=pd.concat([df,df_up_down],axis=1)
df['1-period Positive Money Flow']=df['up']*df['Raw_Money_Flow']
df['1-period Negative Money Flow']=df['down']*df['Raw_Money_Flow']
df['Sum_of_+_14']=df['1-period Positive Money Flow'].rolling(window=14).sum()
df['Sum_of_-_14']=df['1-period Negative Money Flow'].rolling(window=14).sum()
df['MFR']=df['Sum_of_+_14']/df['Sum_of_-_14']
df['MFI']=100-(100/(1+df['MFR']))
df_JPM['MFI']=100-(100/(1+df['MFR']))

In [51]:
df

,Typical Price,Raw_Money_Flow,up,down,1-period Positive Money Flow,1-period Negative Money Flow,Sum_of_+_14,Sum_of_-_14,MFR,MFI
0,97.470001,1.963348e+09,1.0,0.0,1.963348e+09,0.000000e+00,NaN,NaN,NaN,NaN
1,98.943333,3.508580e+09,1.0,0.0,3.508580e+09,0.000000e+00,NaN,NaN,NaN,NaN
2,101.179999,4.169456e+09,1.0,0.0,4.169456e+09,0.000000e+00,NaN,NaN,NaN,NaN
3,102.040000,2.829467e+09,1.0,0.0,2.829467e+09,0.000000e+00,NaN,NaN,NaN,NaN
4,103.660000,4.650675e+09,1.0,0.0,4.650675e+09,0.000000e+00,NaN,NaN,NaN,NaN
5,103.673336,3.432635e+09,1.0,0.0,3.432635e+09,0.000000e+00,NaN,NaN,NaN,NaN
6,104.343333,3.326330e+09,0.0,1.0,0.000000e+00,3.326330e+09,NaN,NaN,NaN,NaN
7,101.586668,4.785707e+09,1.0,0.0,4.785707e+09,0.000000e+00,NaN,NaN,NaN,NaN
8,102.149998,4.047387e+09,1.0,0.0,4.047387e+09,0.000000e+00,NaN,NaN,NaN,NaN
9,103.786665,3.937490e+09,0.0,1.0,0.000000e+00,3.937490e+09,NaN,NaN,NaN,NaN


In [12]:
df['Raw_Money_Flow']=df['Typical Price']*df_JPM['Volume']

In [37]:
up=[]
down=[]
for index,row in df[1:].iterrows():
    if df.iloc[index]['Typical Price'] >= df.iloc[index-1]['Typical Price']:
        up.append(1)
        down.append(0)
      
    else:
        down.append(1)
        up.append(0)


In [30]:
df_up_down=pd.DataFrame(list(zip(up,down)),
              columns=['up','down'])

In [31]:
df_up_down.head()

,up,down
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [32]:
df=pd.concat([df,df_up_down],axis=1)

In [35]:
df.drop(labels=['Up','Down'],axis=1,inplace=True)

In [42]:
df['1-period Positive Money Flow']=df['up']*df['Raw_Money_Flow']
df['1-period Negative Money Flow']=df['down']*df['Raw_Money_Flow']

In [46]:
df['Sum_of_+_14']=df['1-period Positive Money Flow'].rolling(window=14).sum()
df['Sum_of_-_14']=df['1-period Negative Money Flow'].rolling(window=14).sum()

In [48]:
df['MFR']=df['Sum_of_+_14']/df['Sum_of_-_14']

In [50]:
df['MFI']=100-(100/(1+df['MFR']))
df_JPM['MFI']=100-(100/(1+df['MFR']))

In [53]:
df_JPM['MFI']=100-(100/(1+df['MFR']))

In [55]:
df_JPM.to_csv('JPM1.csv')